In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import lightgbm
from sklearn.svm import SVC
from sklearn.utils import shuffle
import pickle
# import main_utils_1
import utils
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
from config2 import SELECTCOLS, ECG_SELECTCOLS, EDA_SELECTCOLS, SELECTFOUR

import config2

In [3]:
len(EDA_SELECTCOLS)

50

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network  import MLPClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [5]:
funcs_for_matbii = [
    (RandomForestClassifier , {'n_estimators': 1000,
                                'min_samples_split': 5,
                                'min_samples_leaf': 1,
                                'max_features': 'auto',
                                'max_depth': 30,
                                'bootstrap': False, 'random_state': 24, 'class_weight': 'balanced'}),
    (LinearDiscriminantAnalysis, {'solver': 'lsqr'}), 
    (GradientBoostingClassifier, {'max_depth': 3, 'n_estimators': 300, 'max_features': 'auto'}), 
    (MLPClassifier, {'hidden_layer_sizes': (100, 10), 'learning_rate': 'adaptive', 'max_iter': 1000}),
    (LogisticRegression, {'C': 1, 'max_iter': 400}),
    (SVC , {'class_weight': 'balanced', 'C': 10}), 
    (xgb.XGBClassifier, {'n_estimators': 300, 'learning_rate': 0.01, 
                         'use_label_encoder': False, 
                         'booster': 'dart', 'n_jobs': 4,
                         'reg_lambda': 0.0001, 'random_state': 24}),
    (lightgbm.LGBMClassifier, {'num_leaves': 100,
                                'n_estimators': 2000,
                                'learning_rate': 0.001,
                                'importance_type': 'gains',
                                'boosting_type': 'gbdt',
                                'class_weight': 'balanced',
                                'random_state': 24})
]

In [6]:
def make_loso(dr_feat_path):
    subjects = os.listdir(dr_feat_path)
    xtrainDriv = pd.DataFrame()
    for subTrain in subjects:
        train = pd.read_csv(os.path.join(dr_feat_path, '{}'.format(subTrain)))
        train[['scrAmpDF_min','scrRecoveryTime_min', 'scrRiseTime_min']].fillna(0)
        if np.isinf(train).values.sum():
            cinf = np.isinf(train).values.sum()
            print("Train Dataframe contains {} values".format(cinf))
        train.replace([np.inf], 9999, inplace=True)        
        train.replace([-np.inf], -9999, inplace=True)        

        train['scrNumPeaks'] = train['scrNumPeaks'].values.astype(int)
        train['scrNumPeaks'] = train['scrNumPeaks'].values.clip(min=0) # converting negatives to zero

        train.dropna(inplace=True)
        xtrainDriv = xtrainDriv.append(train)
        xtrainDriv.reset_index(drop=True, inplace=True)

    return xtrainDriv.copy()

In [7]:
def read_dataset(dataset, folder, basefolder):
    dr_feat_path = r'X:\All Modes\{}\ECG EDA\Combined\{}'.format(dataset, folder) # ECG_EDA_Features_Combined_scld
    bs_feat_path = r'X:\All Modes\{}\ECG EDA\Combined\{}'.format(dataset, basefolder) # ECG_EDA_Base2_Features_Combined

    XtrainDriv = make_loso(dr_feat_path)
    XtrainBase = make_loso(bs_feat_path)

    XtrainDriv = XtrainDriv[SELECTCOLS].copy()
    ytrainDriv = list(XtrainDriv['scaled label'].copy())

    XtrainBase = XtrainBase[SELECTCOLS[:-3]].copy()
    ytrainBase = XtrainBase.shape[0] * [0]

    XtrainDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)
    XtrainDriv = XtrainDriv.append(XtrainBase)

    ytrain = ytrainDriv + ytrainBase

    X = XtrainDriv.values
    
    X, ytrain = shuffle(X, ytrain, random_state=42)

    for idx, val in enumerate(ytrain):
        if val <= 4:
            ytrain[idx] = 0
        else: ytrain[idx] = 1

    return X, ytrain

In [8]:
def mk_training_data(dr_feat_path, sdriv):
    subjects = os.listdir(dr_feat_path)
    xtrainDriv = pd.DataFrame()

    for subTrain in subjects:
        if subTrain != sdriv:
            train = pd.read_csv(os.path.join(dr_feat_path, '{}'.format(subTrain)))

            train[['scrAmpDF_min','scrRecoveryTime_min', 'scrRiseTime_min']].fillna(0)
            if np.isinf(train).values.sum():
                cinf = np.isinf(train).values.sum()
                print("Train Dataframe contains {} values".format(cinf))
            train.replace([np.inf], 9999, inplace=True)        
            train.replace([-np.inf], -9999, inplace=True)        

            train['scrNumPeaks'] = train['scrNumPeaks'].values.astype(int)
            train['scrNumPeaks'] = train['scrNumPeaks'].values.clip(min=0) # converting negatives to zero

            train.dropna(inplace=True)
            xtrainDriv = xtrainDriv.append(train)
            xtrainDriv.reset_index(drop=True, inplace=True)

    return xtrainDriv.copy()

# All Classifiers

In [11]:
def losoValidation(dataset, folder, basefolder, paramsSELECTCOLS):
    dr_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Features_Combined_scld
    bs_feat_path = r'X:\All Modes\{}\ECG EDA\Combined\{}'.format(dataset, basefolder) # ECG_EDA_Base2_Features_Combined
    date_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
    print("Saved Directory: {}".format(date_time))
    savePath_0 = f"X:/All Modes/Data Files/{date_time}"
    utils.mk_dirs(savePath_0)
    savePath1 = os.path.join(savePath_0, f'{dataset}')
    utils.mk_dirs(savePath1)
    savePath = os.path.join(savePath1, 'ECG EDA')
    utils.mk_dirs(savePath)

    mycls = {}

    if dataset == 'MatbII':
        parameter_list = funcs_for_matbii
    # elif dataset == 'Virage':
    #     parameter_list = funcs_for_virage

    subjects = os.listdir(dr_feat_path)
    results_df = pd.DataFrame(columns=['dataset', 'method', 'test_subject', 'test_acc', 'test_f1'])
    for sdriv in subjects:
        utils.mk_dirs(os.path.join(savePath, sdriv))

        xtestDriv = pd.read_csv(os.path.join(dr_feat_path, '{}'.format(sdriv)))
        xtestDriv[['scrAmpDF_min','scrRecoveryTime_min', 'scrRiseTime_min']].fillna(0)
        if np.isinf(xtestDriv).values.sum():
            cinf = np.isinf(xtestDriv).values.sum()
            print("Dataframe contains {} values".format(cinf))

        xtestDriv.replace([np.inf], 9999, inplace=True)
        xtestDriv.replace([-np.inf], -9999, inplace=True)

        xtestDriv['scrNumPeaks'] = xtestDriv['scrNumPeaks'].values.astype(int)
        xtestDriv['scrNumPeaks'] = xtestDriv['scrNumPeaks'].values.clip(min=0) # converting negatives to zero
        xtestDriv.dropna(inplace=True) # .reset_index(drop=True, inplace=True)

        ytestDriv = list(xtestDriv['scaled label'].copy()) 

        XtrainDriv = mk_training_data(dr_feat_path, sdriv)
        # XtrainBase = mk_training_data(bs_feat_path, sdriv)

        XtrainDriv = XtrainDriv[paramsSELECTCOLS].copy()
        XtestDriv = xtestDriv[paramsSELECTCOLS].copy()  ### Look out for small x and X
        # XtrainBase = XtrainBase[SELECTCOLS[:-3]].copy()

        ytrainDriv = list(XtrainDriv['scaled label'].copy())
        # ytrainBase = XtrainBase.shape[0] * [0]

        XtrainDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)
        XtestDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)

        ytrain = ytrainDriv #+ ytrainBase
        # XtrainDriv = XtrainDriv.append(XtrainBase)

        X = XtrainDriv.values
        X, ytrain = shuffle(X, ytrain, random_state=42)

        for idx, val in enumerate(ytrain):
            if val <= 4:
                ytrain[idx] = 0
            else: ytrain[idx] = 1

        for idx, val in enumerate(ytestDriv):
            if val <= 4:
                ytestDriv[idx] = 0
            else: ytestDriv[idx] = 1

        # training different classifier for all subjects and saving them in different dictionnaries
        mycls = {}
        for cls_modl, cls_parameters in parameter_list:
            print("--------------------------------------------")
            print(f"---- Training classifier {cls_modl.__name__} for subject: {sdriv} ---")

            classifier_save_path = os.path.join(savePath, sdriv, cls_modl.__name__)
            utils.mk_dirs(classifier_save_path)

            classifier_report = os.path.join(classifier_save_path, 'report')
            classifier_sav = os.path.join(classifier_save_path, 'classifier')
            utils.mk_dirs(classifier_report)
            utils.mk_dirs(classifier_sav)

            if cls_modl.__name__ in ['LogisticRegression', 'SVC', 'LGBMClassifier', 'XGBClassifier']:
                scaler = StandardScaler()
                X = scaler.fit_transform(X)
                XtestDriv = scaler.transform(XtestDriv)
            clf = cls_modl(**cls_parameters)
            hist = clf.fit(X, ytrain)

            yPred = hist.predict(XtestDriv)

            test_accuray = accuracy_score(ytestDriv, yPred)
            test_f1 = f1_score(ytestDriv, yPred, average='macro')
            
            results_df = results_df.append({'dataset': folder,
                                            'method':cls_modl.__name__,
                                            'test_subject': sdriv,
                                            'test_acc': test_accuray,
                                            'test_f1':test_f1}, ignore_index=True)
            print('Test Subject: {}'.format(sdriv))

            mycls[sdriv] = classification_report(ytestDriv, yPred, zero_division=1, output_dict=True)
            print("----- Classification Report ------")
            print(f"Test accuracy for {sdriv} is: {test_accuray} and f1 score is: {test_f1}\n")

            print(classification_report(ytestDriv, yPred, zero_division=1))
            with open(os.path.join(classifier_report, 'Test_fold_{}_report.pickle'.format(sdriv)), 'wb') as handle:
                pickle.dump(mycls, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(classifier_sav, 'Test_fold_{}_report.sav'.format(sdriv)), 'wb') as handle:
                pickle.dump(hist, handle, protocol= pickle.HIGHEST_PROTOCOL)

        # estimatorPath = os.path.join(savePath, '{}.sav'.format(saveName))
        # pickle.dump(hist, open(estimatorPath, 'wb'))

    results_df.to_csv(os.path.join(savePath, 'results.csv'), index=False)
    return
    # return XtrainDriv, y_train

In [12]:
losoValidation('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined', config2.SELECTEEG)

Saved Directory: 2022_08_10_22_15


C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\3635406624.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xtrainDriv = xtrainDriv.append(train)
C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\3635406624.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xtrainDriv = xtrainDriv.append(train)
C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\3635406624.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xtrainDriv = xtrainDriv.append(train)
C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\3635406624.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xtrainDriv = xtrainDriv.append(train)
C:\Users\Anubhav\AppData\Loc

--------------------------------------------
---- Training classifier RandomForestClassifier for subject: 1105.csv ---


c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\4009010777.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'dataset': folder,


Test Subject: 1105.csv
----- Classification Report ------
Test accuracy for 1105.csv is: 0.5432692307692307 and f1 score is: 0.43515421776291335

              precision    recall  f1-score   support

           0       0.50      0.12      0.19        95
           1       0.55      0.90      0.68       113

    accuracy                           0.54       208
   macro avg       0.52      0.51      0.44       208
weighted avg       0.53      0.54      0.46       208

--------------------------------------------
---- Training classifier LinearDiscriminantAnalysis for subject: 1105.csv ---


...

Test Subject: 1933.csv
----- Classification Report ------
Test accuracy for 1933.csv is: 0.4897959183673469 and f1 score is: 0.43704044117647056

              precision    recall  f1-score   support

           0       0.28      0.25      0.26        36
           1       0.59      0.63      0.61        62

    accuracy                           0.49        98
   macro avg       0.44      0.44      0.44        98
weighted avg       0.48      0.49      0.48        98

--------------------------------------------
---- Training classifier XGBClassifier for subject: 1933.csv ---
[22:45:50] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\4009010777.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'dataset': folder,


Test Subject: 1933.csv
----- Classification Report ------
Test accuracy for 1933.csv is: 0.5612244897959183 and f1 score is: 0.5506983686960231

              precision    recall  f1-score   support

           0       0.43      0.56      0.48        36
           1       0.69      0.56      0.62        62

    accuracy                           0.56        98
   macro avg       0.56      0.56      0.55        98
weighted avg       0.59      0.56      0.57        98

--------------------------------------------
---- Training classifier LGBMClassifier for subject: 1933.csv ---


C:\Users\Anubhav\AppData\Local\Temp\ipykernel_42152\4009010777.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'dataset': folder,


Test Subject: 1933.csv
----- Classification Report ------
Test accuracy for 1933.csv is: 0.5510204081632653 and f1 score is: 0.5387248609328199

              precision    recall  f1-score   support

           0       0.41      0.53      0.46        36
           1       0.67      0.56      0.61        62

    accuracy                           0.55        98
   macro avg       0.54      0.55      0.54        98
weighted avg       0.58      0.55      0.56        98



In [16]:
def losoValidation1(dataset, folder, basefolder, paramsSELECTCOLS):
    dr_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Features_Combined_scld
    bs_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Base2_Features_Combined
    date_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
    print("Saved Directory: {}".format(date_time))
    savePath_0 = f"X:/All Modes/Data Files/{date_time}"
    utils.mk_dirs(savePath_0)
    savePath1 = os.path.join(savePath_0, f'{dataset}')
    utils.mk_dirs(savePath1)
    savePath = os.path.join(savePath1, 'ECG EDA')
    utils.mk_dirs(savePath)

    mycls = {}

    if dataset == 'MatbII':
        parameter_list = funcs_for_matbii
    # elif dataset == 'Virage':
    #     parameter_list = funcs_for_virage

    subjects = os.listdir(dr_feat_path)
    results_df = pd.DataFrame(columns=['dataset', 'method', 'test_subject', 'test_acc', 'test_f1', 'wgt_test_f1'])
    for sdriv in subjects:
        utils.mk_dirs(os.path.join(savePath, sdriv))

        xtestDriv = pd.read_csv(os.path.join(dr_feat_path, '{}'.format(sdriv)))
        xtestDriv[['scrAmpDF_min','scrRecoveryTime_min', 'scrRiseTime_min']].fillna(0)
        if np.isinf(xtestDriv).values.sum():
            cinf = np.isinf(xtestDriv).values.sum()
            print("Dataframe contains {} values".format(cinf))

        xtestDriv.replace([np.inf], 9999, inplace=True)
        xtestDriv.replace([-np.inf], -9999, inplace=True)

        xtestDriv['scrNumPeaks'] = xtestDriv['scrNumPeaks'].values.astype(int)
        xtestDriv['scrNumPeaks'] = xtestDriv['scrNumPeaks'].values.clip(min=0) # converting negatives to zero
        xtestDriv.dropna(inplace=True) # .reset_index(drop=True, inplace=True)

        ytestDriv = list(xtestDriv['scaled label'].copy()) 

        XtrainDriv = mk_training_data(dr_feat_path, sdriv)
        # XtrainBase = mk_training_data(bs_feat_path, sdriv)

        XtrainDriv = XtrainDriv[paramsSELECTCOLS].copy()
        XtestDriv = xtestDriv[paramsSELECTCOLS].copy()  ### Look out for small x and X
        # XtrainBase = XtrainBase[SELECTCOLS[:-3]].copy()

        ytrainDriv = list(XtrainDriv['scaled label'].copy())
        # ytrainBase = XtrainBase.shape[0] * [0]

        XtrainDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)
        XtestDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)

        ytrain = ytrainDriv #+ ytrainBase
        # XtrainDriv = XtrainDriv.append(XtrainBase)

        X = XtrainDriv.values
        X, ytrain = shuffle(X, ytrain, random_state=42)

        for idx, val in enumerate(ytrain):
            if val <= 4:
                ytrain[idx] = 0
            else: ytrain[idx] = 1

        for idx, val in enumerate(ytestDriv):
            if val <= 4:
                ytestDriv[idx] = 0
            else: ytestDriv[idx] = 1

        # training different classifier for all subjects and saving them in different dictionnaries
        mycls = {}
        for cls_modl, cls_parameters in parameter_list:
            print("--------------------------------------------")
            print(f"---- Training classifier {cls_modl.__name__} for subject: {sdriv} ---")

            classifier_save_path = os.path.join(savePath, sdriv, cls_modl.__name__)
            utils.mk_dirs(classifier_save_path)

            classifier_report = os.path.join(classifier_save_path, 'report')
            classifier_sav = os.path.join(classifier_save_path, 'classifier')
            utils.mk_dirs(classifier_report)
            utils.mk_dirs(classifier_sav)

            if cls_modl.__name__ in ['LogisticRegression', 'SVC', 'LGBMClassifier', 'XGBClassifier']:
                scaler = StandardScaler()
                X = scaler.fit_transform(X)
                XtestDriv = scaler.transform(XtestDriv)
            clf = cls_modl(**cls_parameters)
            hist = clf.fit(X, ytrain)

            yPred = hist.predict(XtestDriv)

            test_accuray = accuracy_score(ytestDriv, yPred)
            test_f1 = f1_score(ytestDriv, yPred, average='macro')
            wgt_test_f1 = f1_score(ytestDriv, yPred, average='weighted')

            results_df = results_df.append({'dataset': folder,
                                            'method':cls_modl.__name__,
                                            'test_subject': sdriv,
                                            'test_acc': test_accuray,
                                            'test_f1':test_f1, 'wgt_test_f1': wgt_test_f1}, ignore_index=True)
            print('Test Subject: {}'.format(sdriv))

            mycls[sdriv] = classification_report(ytestDriv, yPred, zero_division=1, output_dict=True)
            print("----- Classification Report ------")
            print(f"Test accuracy for {sdriv} is: {test_accuray} and f1 score is: {test_f1}\n")

            print(classification_report(ytestDriv, yPred, zero_division=1))
            with open(os.path.join(classifier_report, 'Test_fold_{}_report.pickle'.format(sdriv)), 'wb') as handle:
                pickle.dump(mycls, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(classifier_sav, 'Test_fold_{}_report.sav'.format(sdriv)), 'wb') as handle:
                pickle.dump(hist, handle, protocol= pickle.HIGHEST_PROTOCOL)

        # estimatorPath = os.path.join(savePath, '{}.sav'.format(saveName))
        # pickle.dump(hist, open(estimatorPath, 'wb'))

    results_df.to_csv(os.path.join(savePath, 'results.csv'), index=False)
    return
    # return XtrainDriv, y_train

In [15]:
losoValidation1('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined', config2.SELECTEEG)

Saved Directory: 2022_08_10_22_52
--------------------------------------------
---- Training classifier RandomForestClassifier for subject: 1105.csv ---
Test Subject: 1105.csv
----- Classification Report ------
Test accuracy for 1105.csv is: 0.5432692307692307 and f1 score is: 0.43515421776291335

              precision    recall  f1-score   support

           0       0.50      0.12      0.19        95
           1       0.55      0.90      0.68       113

    accuracy                           0.54       208
   macro avg       0.52      0.51      0.44       208
weighted avg       0.53      0.54      0.46       208

--------------------------------------------
---- Training classifier LinearDiscriminantAnalysis for subject: 1105.csv ---
Test Subject: 1105.csv
----- Classification Report ------
Test accuracy for 1105.csv is: 0.40865384615384615 and f1 score is: 0.40699534107502955

              precision    recall  f1-score   support

           0       0.36      0.39      0.38      

In [21]:
cols_run = {"ECG_EEG": config2.SELECT_ECG_EEG, "ECG_GAZE": config2.SELECT_ECG_GAZE, "EDA_EEG": config2.SELECT_EDA_EEG,
            "EDA_GAZE": config2.SELECT_EDA_GAZE, "EEG_GAZE": config2.SELECT_EEG_GAZE,
            "ECG_EDA_EEG": config2.SELECT_ECG_EDA_EEG, "ECG_EDA_GAZE": config2.SELECT_ECG_EDA_GAZE,
            "ECG_EEG_GAZE": config2.SELECT_ECG_EEG_GAZE, "EDA_EEG_GAZE": config2.SELECT_EDA_EEG_GAZE}

In [23]:
for name, mods in cols_run.items():
    print(f"----- Running Modality Combination {name}")
    losoValidation1('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined', mods)


----- Running Modality Combination ECG_EEG
Saved Directory: 2022_08_11_14_49
--------------------------------------------
---- Training classifier RandomForestClassifier for subject: 1105.csv ---
Test Subject: 1105.csv
----- Classification Report ------
Test accuracy for 1105.csv is: 0.5673076923076923 and f1 score is: 0.4155114275009367

              precision    recall  f1-score   support

           0       0.86      0.06      0.12        95
           1       0.56      0.99      0.71       113

    accuracy                           0.57       208
   macro avg       0.71      0.53      0.42       208
weighted avg       0.69      0.57      0.44       208

--------------------------------------------
---- Training classifier LinearDiscriminantAnalysis for subject: 1105.csv ---
Test Subject: 1105.csv
----- Classification Report ------
Test accuracy for 1105.csv is: 0.5192307692307693 and f1 score is: 0.44956070710278395

              precision    recall  f1-score   support

        

In [22]:
for key, value in cols_run.items():
    print(key)

ECG_EEG
ECG_GAZE
EDA_EEG
EDA_GAZE
EEG_GAZE
ECG_EDA_EEG
ECG_EDA_GAZE
ECG_EEG_GAZE
EDA_EEG_GAZE


In [9]:
def mk_kfold_data(dr_feat_path):
    subjects = os.listdir(dr_feat_path)
    xtrainDriv = pd.DataFrame()

    for subTrain in subjects:
        # if subTrain != sdriv:
        train = pd.read_csv(os.path.join(dr_feat_path, '{}'.format(subTrain)))

        train[['scrAmpDF_min','scrRecoveryTime_min', 'scrRiseTime_min']].fillna(0)
        if np.isinf(train).values.sum():
            cinf = np.isinf(train).values.sum()
            print("Train Dataframe contains {} values".format(cinf))
        train.replace([np.inf], 9999, inplace=True)        
        train.replace([-np.inf], -9999, inplace=True)        

        train['scrNumPeaks'] = train['scrNumPeaks'].values.astype(int)
        train['scrNumPeaks'] = train['scrNumPeaks'].values.clip(min=0) # converting negatives to zero

        train.dropna(inplace=True)
        xtrainDriv = xtrainDriv.append(train)
        xtrainDriv.reset_index(drop=True, inplace=True)

    return xtrainDriv.copy()

In [15]:
def kfoldValidation1(dataset, folder, basefolder, paramsSELECTCOLS):
    dr_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Features_Combined_scld
    bs_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Base2_Features_Combined
    date_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
    print("Saved Directory: {}".format(date_time))
    savePath_0 = f"X:/All Modes/Data Files/{date_time}"
    utils.mk_dirs(savePath_0)
    savePath1 = os.path.join(savePath_0, f'{dataset}')
    utils.mk_dirs(savePath1)
    savePath = os.path.join(savePath1, 'ECG EDA')
    utils.mk_dirs(savePath)

    mycls = {}

    if dataset == 'MatbII':
        parameter_list = funcs_for_matbii

    results_df = pd.DataFrame(columns=['dataset', 'method', 'test_subject', 'test_acc', 'test_f1', 'wgt_test_f1'])
    XtrainDriv = mk_kfold_data(dr_feat_path)
    XtrainDriv = XtrainDriv[paramsSELECTCOLS].copy()
    ytrainDriv = list(XtrainDriv['scaled label'].copy())

    XtrainDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)
    ytrain = ytrainDriv #+ ytrainBase

    X = XtrainDriv.values
    X, ytrain = shuffle(X, ytrain, random_state=42)

    for idx, val in enumerate(ytrain):
        if val <= 4:
            ytrain[idx] = 0
        else: ytrain[idx] = 1


    tenFoldSplit = KFold(n_splits=10)
    counter = 1

    for train_index, test_index in tenFoldSplit.split(X):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = np.asarray(ytrain)[train_index], np.asarray(ytrain)[test_index]
        print(counter)

        # training different classifier for all subjects and saving them in different dictionnaries
        mycls = {}
        for cls_modl, cls_parameters in parameter_list:
            print("--------------------------------------------")
            print(f"---- Training classifier {cls_modl.__name__} for fold: {counter} ---")

            classifier_save_path = os.path.join(savePath, str(counter), cls_modl.__name__)
            utils.mk_dirs(classifier_save_path)

            classifier_report = os.path.join(classifier_save_path, 'report')
            classifier_sav = os.path.join(classifier_save_path, 'classifier')
            utils.mk_dirs(classifier_report)
            utils.mk_dirs(classifier_sav)

            if cls_modl.__name__ in ['LogisticRegression', 'SVC', 'LGBMClassifier', 'XGBClassifier']:
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
            clf = cls_modl(**cls_parameters)
            hist = clf.fit(X_train, y_train)

            yPred = hist.predict(X_test)

            test_accuray = accuracy_score(y_test, yPred)
            test_f1 = f1_score(y_test, yPred, average='macro')
            wgt_test_f1 = f1_score(y_test, yPred, average='weighted')

            results_df = results_df.append({'dataset': folder,
                                            'method':cls_modl.__name__,
                                            'test_subject': counter,
                                            'test_acc': test_accuray,
                                            'test_f1':test_f1, 'wgt_test_f1': wgt_test_f1}, ignore_index=True)
            print('Test Subject: {}'.format(counter))

            mycls[counter] = classification_report(y_test, yPred, zero_division=1, output_dict=True)
            print("----- Classification Report ------")
            print(f"Test accuracy for {counter} is: {test_accuray} and f1 score is: {test_f1}\n")

            print(classification_report(y_test, yPred, zero_division=1))
            with open(os.path.join(classifier_report, 'Test_fold_{}_report.pickle'.format(counter)), 'wb') as handle:
                pickle.dump(mycls, handle, protocol= pickle.HIGHEST_PROTOCOL)

            with open(os.path.join(classifier_sav, 'Test_fold_{}_report.sav'.format(counter)), 'wb') as handle:
                pickle.dump(hist, handle, protocol= pickle.HIGHEST_PROTOCOL)

        counter += 1
    results_df.to_csv(os.path.join(savePath, 'results.csv'), index=False)
    return

In [11]:
cols_run_ALL = {"ECG": config2.SELECTECG, "EDA": config2.SELECTEDA, "EEG": config2.SELECTEEG, "GAZE": config2.SELECTGAZE,
            "ECG_EEG": config2.SELECT_ECG_EEG, "ECG_GAZE": config2.SELECT_ECG_GAZE,
            "EDA_EEG": config2.SELECT_EDA_EEG, "EDA_GAZE": config2.SELECT_EDA_GAZE,
            "EEG_GAZE": config2.SELECT_EEG_GAZE,
            "ECG_EDA_EEG": config2.SELECT_ECG_EDA_EEG, "ECG_EDA_GAZE": config2.SELECT_ECG_EDA_GAZE,
            "ECG_EEG_GAZE": config2.SELECT_ECG_EEG_GAZE, "EDA_EEG_GAZE": config2.SELECT_EDA_EEG_GAZE}

In [16]:
for name, mods in cols_run_ALL.items():
    print(f"----- Running Modality Combination {name}")
    kfoldValidation1('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined', mods)

----- Running Modality Combination ECG
Saved Directory: 2022_08_12_00_03
1
--------------------------------------------
---- Training classifier RandomForestClassifier for fold: 1 ---
Test Subject: 1
----- Classification Report ------
Test accuracy for 1 is: 0.6949152542372882 and f1 score is: 0.68890607424072

              precision    recall  f1-score   support

           0       0.67      0.63      0.65       131
           1       0.72      0.75      0.73       164

    accuracy                           0.69       295
   macro avg       0.69      0.69      0.69       295
weighted avg       0.69      0.69      0.69       295

--------------------------------------------
---- Training classifier LinearDiscriminantAnalysis for fold: 1 ---
Test Subject: 1
----- Classification Report ------
Test accuracy for 1 is: 0.6203389830508474 and f1 score is: 0.6009661835748792

              precision    recall  f1-score   support

           0       0.60      0.45      0.51       131
       

In [17]:
cols_run_two = {"ECG_EDA_EEG_GAZE": config2.SELECTFOUR, "ECG_EDA": config2.SELECTECGEDA}

for name, mods in cols_run_two.items():
    print(f"----- Running Modality Combination {name}")
    kfoldValidation1('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined', mods)

----- Running Modality Combination ECG_EDA_EEG_GAZE
Saved Directory: 2022_08_12_10_39
1
--------------------------------------------
---- Training classifier RandomForestClassifier for fold: 1 ---
Test Subject: 1
----- Classification Report ------
Test accuracy for 1 is: 0.7796610169491526 and f1 score is: 0.7751023328368187

              precision    recall  f1-score   support

           0       0.77      0.72      0.74       131
           1       0.79      0.83      0.81       164

    accuracy                           0.78       295
   macro avg       0.78      0.77      0.78       295
weighted avg       0.78      0.78      0.78       295

--------------------------------------------
---- Training classifier LinearDiscriminantAnalysis for fold: 1 ---
Test Subject: 1
----- Classification Report ------
Test accuracy for 1 is: 0.6847457627118644 and f1 score is: 0.6775728942636534

              precision    recall  f1-score   support

           0       0.66      0.60      0.63   